**Recomendação Por Título**

In [ ]:
import pandas as pd
import re
import unicodedata
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import math
import nltk
from nltk.corpus import stopwords

# Baixar stopwords
nltk.download('stopwords')

# Lista de stopwords em português
stop_words = set(stopwords.words('portuguese'))

# Função para limpar os títulos das músicas
def preprocess_song_name(song_name):
    if pd.isna(song_name):
        return ""

    title = re.sub(r"[\[\]']", "", song_name)
    title = re.sub(r"[^\w\s]", "", title)
    title = title.lower()
    title = ''.join(
        char for char in unicodedata.normalize('NFD', title)
        if unicodedata.category(char) != 'Mn'
    )

    # Remove stop words
    words = title.split()
    filtered_words = [word for word in words if word not in stop_words]

    return " ".join(filtered_words)

# Função de recomendação
def recommend_songs(user_input, songs_df):
    user_input = preprocess_song_name(user_input)
    user_vector = np.zeros(len(unique_words))

    for word in user_input.split():
        if word in word_index:
            user_vector[word_index[word]] = 1

    similarities = cosine_similarity([user_vector], word_bow.values)[0]
    angles = [round(math.degrees(math.acos(sim)), 1) if -1 <= sim <= 1 else None for sim in similarities]

    songs_df['similarity'] = similarities
    songs_df['angle_degrees'] = angles

    recommended_songs = songs_df.sort_values(by='similarity', ascending=False).head(10)
    return recommended_songs[['song_name', 'artist']]

# Carregando o dataset


data_path = '/content/bossa_nova_songs_dataset.xlsx'
songs_df = pd.read_excel(data_path)


# Limpando os nomes das músicas
songs_df['cleaned_song_name'] = songs_df['song_name'].apply(preprocess_song_name)

# Criando o índice de palavras únicas
unique_words = sorted(set(" ".join(songs_df['cleaned_song_name']).split()))
word_index = {word: idx for idx, word in enumerate(unique_words)}

# Criando a matriz Bag of Words
word_bow = pd.DataFrame(0, index=songs_df.index, columns=unique_words)
for idx, song in enumerate(songs_df['cleaned_song_name']):
    for word in song.split():
        if word in word_bow.columns:
            word_bow.loc[idx, word] = 1

# Entrada do usuário e recomendação
user_input = str(input("Digite o nome da música para recomendar: "))
recommendations = recommend_songs(user_input, songs_df)

# Exibindo somente nomes e artistas recomendados
for idx, row in recommendations.iterrows():
    print(f"{row['song_name']} - {row['artist']}")




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


**Recomendação por Letra**

In [ ]:
import pandas as pd
import re
import unicodedata
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import math
import nltk
from nltk.corpus import stopwords

# Baixando a lista de stopwords
nltk.download('stopwords')
stop_words = set(stopwords.words('portuguese'))  # Utilizando stopwords em português

def preprocess_lyrics(lyrics):
    if pd.isna(lyrics):
        return ""
    lyrics = lyrics.lower()  # Transforma tudo em minúsculo
    lyrics = re.sub(r'[^\w\s]', '', lyrics)  # Remove pontuações
    lyrics = ''.join(char for char in unicodedata.normalize('NFD', lyrics) if unicodedata.category(char) != 'Mn')  # Remove acentos
    lyrics = ' '.join(word for word in lyrics.split() if word not in stop_words)  # Remove stopwords
    return lyrics

def recommend_songs(user_input, songs_df):
    user_input = preprocess_lyrics(user_input)  # Pré-processa a entrada do usuário

    all_lyrics = songs_df['cleaned_song_lyrics'].tolist() + [user_input]  # Junta as letras das músicas com a entrada do usuário

    vectorizer = CountVectorizer(binary=True)  #  (bag of words)
    bow_matrix = vectorizer.fit_transform(all_lyrics)

    similarities = cosine_similarity(bow_matrix[-1], bow_matrix[:-1])[0]  # Calcula a similaridade de cosseno
    angles = [round(math.degrees(math.acos(sim)), 1) if -1 <= sim <= 1 else None for sim in similarities]

    songs_df['similarity'] = similarities  # Adiciona a similaridade no dataframe
    songs_df['angle_degrees'] = angles  # Adiciona o ângulo em graus no dataframe

    recommended_songs = songs_df.sort_values(by='similarity', ascending=False).head(10)  # Seleciona as 10 músicas mais similares
    return recommended_songs[['song_name', 'artist', 'song_lyrics']]  # Retorna as colunas desejadas

# Carregamento dos dados
data_path = '/content/bossa_nova_songs_dataset.xlsx'
songs_df = pd.read_excel(data_path)

# Limpeza das letras das músicas
songs_df['cleaned_song_lyrics'] = songs_df['song_lyrics'].apply(preprocess_lyrics)

# Entrada do usuário
user_input = str(input("Digite a letra ou parte da letra da música para recomendar: "))
recommendations = recommend_songs(user_input, songs_df)

# Exibindo as recomendações (título, artista e letra completa)
print("Músicas recomendadas:")
for index, row in recommendations.iterrows():
    print(f"Título: {row['song_name']}")
    print(f"Artista: {row['artist']}")
    print(f"Letra:\n{row['song_lyrics']}\n")



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Digite a letra ou parte da letra da música para recomendar: amor
Músicas recomendadas:
Título:  Refúgio  
Artista:   Wanda Sá 
Letra:
  Sob Suas asas de amor eu me refugio ó meu Senhor Somente em Ti confio Somente em Ti meu Senhor  

Título:  Nega do Cabelo Duro  
Artista:   Astrud Gilberto 
Letra:
  Nega do cabelo duro, Qual é o pente que te penteia? Qual é o pente que te penteia? Qual é o pente que te penteia? Nega do cabelo duro, Qual é o pente que te penteia? Qual é o pente que te penteia? Qual é o pente que te penteia? Teu cabelo está na moda, o teu corpo bamboleia. Minha nega, meu amor, Qual é o pente que te penteia? Nega do cabelo duro, Qual é o pente que te penteia? Qual é o pente que te penteia? Qual é o pente que te penteia?  

Título:  Acima Das Estrelas  
Artista:   Wanda Sá 
Letra:
  Salvou-me Salvou-me Salvou-me Pelo Seu amor O Homem que mora que mora acima das estrelas Ele mora acima das estrelas Ele é Jesus o Rei da glória Coroou-me Coroou-me Coroou-me Pelo Seu amor Amo